# I94 Database - Exploration Notebook

In [ ]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession

path_immigration = "output/fact/*"
spark = SparkSession.builder.enableHiveSupport().getOrCreate()
df = spark.read.parquet(path_immigration)
df.createOrReplaceTempView("df")

df.head()

### Query 1: Filtered for teenagers and sorted by total people

In [ ]:
query = spark.sql("""
    SELECT
        df.month,
        df.state,
        df.city,
        df.origin_country,
        df.gender,
        df.age,
        SUM(df.total_flights) AS total_flights,
        SUM(df.total_people) AS total_people,
        SUM(df.total_people)/SUM(df.total_flights) AS avg_per_flight
    FROM
        df
    WHERE
        df.month = 4 AND
        df.age < 19 AND
        df.age > 12 AND
        df.mode_transport = "Air"
    GROUP BY
        df.month,
        df.state,
        df.city,
        df.origin_country,
        df.gender,
        df.age
    ORDER BY
        SUM(df.total_people) DESC
""")

print(query.sort(query.total_people.desc()).show(25, truncate=True))
# query.createOrReplaceTempView("query1")

### Query 2: Specific avg number of passengers per flight (estimated)

In [ ]:

query.createOrReplaceTempView("query1")
query = spark.sql("""
    SELECT
        query1.*
    FROM
        query1
    WHERE
        query1.avg_per_flight > 5
""")

print(query.sort(query.total_people.desc()).show(5, truncate=True))
query.createOrReplaceTempView("query2")